In [1]:
import ee
import geemap
import os

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
site = ee.FeatureCollection('users/Introduction/study_area')
Map.addLayer(site, {}, "study_area")

In [5]:
Map.centerObject(site, 9)

modis = ee.ImageCollection("MODIS/006/MOD13Q1").select('NDVI')

img = modis.filter(ee.Filter.calendarRange(2001,2020,'year')) \
.filter(ee.Filter.calendarRange(2,2,'month'))

# Scale the imagecollection and chart time series

def func_nfr(image):
  return image.multiply(0.0001) \
    .copyProperties(image, ["system:time_start", "system:time_end"])

ndvi =  img.map(func_nfr)


# reduce image collection with mean()
ndvi = ndvi.mean()

# Set visualization parameters


# add layer
Map.addLayer(ndvi.clip(site), {'palette': ["red", "yellow", "green"], 'min': 0.1, 'max': 0.8}, "NDVI")

# 3. Classify NDVI into 5 classes
ndvi2 = ee.Image(0) \
          .where(ndvi.gt(0.0).And(ndvi.lte(0.1)), 1) \
          .where(ndvi.gt(0.1).And(ndvi.lte(0.2)), 2) \
          .where(ndvi.gt(0.2).And(ndvi.lte(0.3)), 3) \
          .where(ndvi.gt(0.3).And(ndvi.lte(0.4)), 4) \
          .where(ndvi.gt(0.4).And(ndvi.lte(0.5)), 5) \
          .where(ndvi.gt(0.5).And(ndvi.lte(0.6)), 6) \
          .where(ndvi.gt(0.6).And(ndvi.lte(0.7)), 7) \
          .where(ndvi.gt(0.7).And(ndvi.lte(0.8)), 8) \
          .where(ndvi.gt(0.8), 9)

ndvi2 = ndvi2.clip(site)
# Add map layers
Map.addLayer(ndvi2, {'min': 1, 'max': 9, 'palette': ["#d73027", "#f46d43", "#fdae61", "#fee08b", "#ffffbf", "#d9ef8b", "#a6d96a", "#66bd63", "#1a9850", "#006837"]}, 'Classified NDVI',True)


In [6]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, '2_ndvi_Feb.tif')

In [7]:
geemap.ee_export_image(ndvi2, filename=filename, scale=250, region=site.geometry(), file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Mukesh\Downloads\2_ndvi_Feb.tif
